# Chicago Public Official Overtime Analysis 

This notebook pulls data from the Chicago Data Portal and persists it in a SQLite Database

This work is split into an additional notebook.  This data was also analyzed with Airbnb's Superset

### Load dependencies

In [7]:
import pandas as pd
import sqlite3


---

## Data ingestion
---

### Download Links:

In [12]:
ot_download_links = {
    2016:"https://data.cityofchicago.org/api/views/tsqz-67gi/rows.csv?accessType=DOWNLOAD",
    2015:"https://data.cityofchicago.org/api/views/efd5-hyfi/rows.csv?accessType=DOWNLOAD",
    2014:"https://data.cityofchicago.org/api/views/9xua-tabs/rows.csv?accessType=DOWNLOAD",
    2013:"https://data.cityofchicago.org/api/views/s9zb-nuc6/rows.csv?accessType=DOWNLOAD",
    2012:"https://data.cityofchicago.org/api/views/92xk-4rg9/rows.csv?accessType=DOWNLOAD"
}

In [13]:
crime_download_links = {
    2016:"https://data.cityofchicago.org/api/views/kf95-mnd6/rows.csv?accessType=DOWNLOAD",
    2015:"https://data.cityofchicago.org/api/views/vwwp-7yr9/rows.csv?accessType=DOWNLOAD",
    2014:"https://data.cityofchicago.org/api/views/qnmj-8ku6/rows.csv?accessType=DOWNLOAD",
    2013:"https://data.cityofchicago.org/api/views/a95h-gwzm/rows.csv?accessType=DOWNLOAD",
    2012:"https://data.cityofchicago.org/api/views/hx8q-mf9v/rows.csv?accessType=DOWNLOAD"
}


### Download data from each url - store as Pandas DataFrames

Stored as dictionary:

`{df_name: DataFrame}`

This is one way to work with multiple dataframes

In [14]:
ot_data = {} #overtime data frames
for year in ot_download_links.keys():
    ot_data['ot_'+ str(year)] = pd.read_csv(ot_download_links[year])
    print("Shape of OT data for "+ str(year)+": " + str(ot_data['ot_'+ str(year)].shape))

Shape of OT data for 2016: (25764, 16)
Shape of OT data for 2012: (24270, 16)
Shape of OT data for 2013: (22708, 16)
Shape of OT data for 2014: (24273, 16)
Shape of OT data for 2015: (24082, 16)


In [15]:
crime_data = {}
for year in crime_download_links.keys():
    crime_data['crime_'+ str(year)] = pd.read_csv(crime_download_links[year])
    print("Shape of crime data for "+ str(year)+ ": "+ str(crime_data['crime_'+ str(year)].shape))

Shape of crime data for 2016: (267453, 22)
Shape of crime data for 2012: (335738, 22)
Shape of crime data for 2013: (306761, 22)
Shape of crime data for 2014: (274666, 22)
Shape of crime data for 2015: (263231, 22)


# Overtime Data 
---


In [74]:
ot_data['ot_2012'].head(1)

,dept_name,emp_name,title,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,total,year
0,Administrative Hearings,"WOODRIDGE, RACHENETTE",ADMINISTRATIVE ASST II,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,445.0,0.0,445.0,2012


#### Check out headers

This is the same data set from 5 years, I wanted to see if the schema was consistent.  The naming conventions changed, but the order of column values was consistent over the overs.  

- If there are more years to analyze, write a function to compare column names and/or dtypes to see if schema changes over the years

In [75]:
for file_name in ot_data.keys():
    print( ot_data[file_name].columns)

Index([u'dept_name', u'emp_name', u'title', u'jan', u'feb', u'mar', u'apr',
       u'may', u'jun', u'jul', u'aug', u'sep', u'oct', u'nov', u'dec',
       u'total', u'year'],
      dtype='object')
Index([u'dept_name', u'emp_name', u'title', u'jan', u'feb', u'mar', u'apr',
       u'may', u'jun', u'jul', u'aug', u'sep', u'oct', u'nov', u'dec',
       u'total', u'year'],
      dtype='object')
Index([u'dept_name', u'emp_name', u'title', u'jan', u'feb', u'mar', u'apr',
       u'may', u'jun', u'jul', u'aug', u'sep', u'oct', u'nov', u'dec',
       u'total', u'year'],
      dtype='object')
Index([u'dept_name', u'emp_name', u'title', u'jan', u'feb', u'mar', u'apr',
       u'may', u'jun', u'jul', u'aug', u'sep', u'oct', u'nov', u'dec',
       u'total', u'year'],
      dtype='object')
Index([u'dept_name', u'emp_name', u'title', u'jan', u'feb', u'mar', u'apr',
       u'may', u'jun', u'jul', u'aug', u'sep', u'oct', u'nov', u'dec',
       u'total', u'year'],
      dtype='object')


### Clean data

- Found the numbers were stored as strings.  
- Convert the strings to floats, don't convert the actual string columns, print the name of the column if can't convert for some reason.

In [76]:
def ot_clean(month_df):
    for col in month_df.columns.tolist():
        if col.upper() not in ['DEPARTMENT NAME', 'EMPLOYEE NAME', 'TITLE', 'NAME']:
            try:
                month_df[col] = month_df[col].str.replace(',', '')
                month_df[col] = month_df[col].str.replace('$', '')
                month_df[col] = month_df[col].astype(float)
            except: 
                print(col)
    return(month_df)

In [77]:
for file_name in ot_data.keys():
    try:
        ot_data[file_name] = ot_clean(ot_data[file_name])
    except:
        print (file_name)

dept_name
emp_name
jan
feb
mar
apr
may
jun
jul
aug
sep
oct
nov
dec
total
year
dept_name
emp_name
jan
feb
mar
apr
may
jun
jul
aug
sep
oct
nov
dec
total
year
dept_name
emp_name
jan
feb
mar
apr
may
jun
jul
aug
sep
oct
nov
dec
total
year
dept_name
emp_name
jan
feb
mar
apr
may
jun
jul
aug
sep
oct
nov
dec
total
year
dept_name
emp_name
jan
feb
mar
apr
may
jun
jul
aug
sep
oct
nov
dec
total
year


### Add a column for the year for storage

In [78]:
from datetime import datetime

#### Fixing my mistake - found when investigating DB data.

In [87]:
# for file_name in ot_data.keys():
#     ot_data[file_name].drop(['Year','year'], axis=1, inplace=True)

In [89]:
for file_name in ot_data.keys():
    ot_data[file_name]['year'] = datetime.strptime('12/31/'+ str(file_name[-4:]), '%m/%d/%Y')

### Create DB + Add our data

In [90]:
def create_db(db_path):
    '''Bad function design'''
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    try:    
        cur.execute("DROP TABLE IF EXISTS ot")#drop if already exists tables in DB
        cur.execute("""CREATE TABLE ot
           (dept_name text, emp_name text, title text, jan numeric, feb numeric,
       mar numeric, apr numeric, may numeric, jun numeric,jul numeric, aug numeric, sep numeric,
       oct numeric, nov numeric, dec numeric, total numeric, year text)
        """)
    except Exception as e:
        print(e)
    con.commit()
    con.close()

Need consistent column names for all the pandas dataframes.  We'll be pushing dataframes directly to sqlite.


In [91]:
table_cols = ['dept_name' , 'emp_name', 'title' , 'jan' , 'feb' ,'mar' , 'apr' , 'may' , 'jun' ,'jul' , 'aug' , 'sep' , 'oct' , 'nov' , 'dec' , 'total' , 'year']

In [92]:
db_path = 'chicago_data.db'
create_db(db_path)

In [93]:
con = sqlite3.connect(db_path)#Create database to store movie and rating info for easy storage and querying
cur = con.cursor()

#### One last check on the data before adding to DB

In [94]:
ot_data['ot_2012'].head()

,dept_name,emp_name,title,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,total,year
0,Administrative Hearings,WOODRIDGE RACHENETTE,ADMINISTRATIVE ASST II,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,445.0,0.0,445.0,2012-12-31
1,Animal Care and Control,HOLLOWAY EFREM J,ANIMAL CARE AIDE II,278.0,670.0,1040.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,287.0,2275.0,2012-12-31
2,Animal Care and Control,EDGECOMBE CHERYL K,ANIMAL CONTROL OFFICER,404.0,0.0,771.0,0.0,147.0,637.0,1114.0,478.0,257.0,539.0,257.0,514.0,5118.0,2012-12-31
3,Animal Care and Control,RUGLIC MARY S,ANIMAL CONTROL OFFICER,0.0,147.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,147.0,2012-12-31
4,Animal Care and Control,WASHINGTON MONTESE DAJUAN,VETERINARY ASST,575.0,780.0,1135.0,492.0,657.0,913.0,1552.0,821.0,780.0,954.0,855.0,472.0,9985.0,2012-12-31


#### Loop through my dictionary keys, using:    `Pandas.DataFrame.to_sql(table_name, connection, ...)`

In [95]:
for file_name in ot_data.keys():
    ot_data[file_name].columns = table_cols
    ot_data[file_name].to_sql('ot', con, if_exists='append', index=False)

In [96]:
cur.execute("""SELECT COUNT(*) FROM ot""")

In [97]:
cur.fetchall()

[(121097,)]

### 'ot' Table Column Names:

- 'dept_name'
- 'emp_name'
- 'title'
- 'jan'
- 'feb'
- 'mar'
- 'apr'
- 'may'
- 'jun'
- 'jul'
- 'aug' 
- 'sep'
- 'oct'
- 'nov'
- 'dec'
- 'total' 
- 'year'

In [28]:
cur.execute("""SELECT total, year 
                FROM ot
                GROUP BY year""")

In [29]:
q = """SELECT total, year 
                FROM ot
                GROUP BY year"""

In [30]:
df = pd.read_sql(q,con)

In [31]:
df.head()

,total,year
0,15554.00,2012
1,609.30,2013
2,50211.49,2014
3,22773.31,2015
4,25249.60,2016


In [32]:
cur.close()

Dollar values are looking like strings, we'll need to fix that.  I also don't care about about names for now, more concerned with Department Name.  Title could be interesting for other analysis but lets start with analysis grouped by Department.

---

# Crime Data - check schema
---

In [33]:
crime_data['crime_2012'].describe()

/Users/gregfilla/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,ID,Beat,District,Ward,X Coordinate,Community Area,Y Coordinate,Year,Latitude,Longitude
count,3.357380e+05,335738.000000,335738.000000,335731.000000,3.347490e+05,335712.000000,3.347490e+05,335738.0,334749.000000,334749.000000
mean,8.684269e+06,1164.185115,11.334130,22.834299,1.164500e+06,37.601974,1.885368e+06,2012.0,41.841044,-87.671861
std,3.781077e+05,694.436440,6.918046,13.784707,1.882923e+04,21.562858,3.503073e+04,0.0,0.096473,0.067633
min,2.022400e+04,111.000000,1.000000,1.000000,0.000000e+00,0.000000,0.000000e+00,2012.0,36.619446,-91.686566
25%,8.560693e+06,614.000000,6.000000,NaN,NaN,NaN,NaN,2012.0,NaN,NaN
50%,8.692700e+06,1024.000000,10.000000,NaN,NaN,NaN,NaN,2012.0,NaN,NaN
75%,8.825174e+06,1722.000000,17.000000,NaN,NaN,NaN,NaN,2012.0,NaN,NaN
max,1.095699e+07,2535.000000,31.000000,50.000000,1.205119e+06,77.000000,1.951527e+06,2012.0,42.022586,-87.524529


In [34]:
for file_name in crime_data.keys():
    print( crime_data[file_name].shape)   

(335738, 22)
(306761, 22)
(267453, 22)
(274666, 22)
(263231, 22)


In [99]:
crime_data['crime_2012'].columns

Index([u'ID', u'Case_Number', u'Date', u'Block', u'IUCR', u'Primary_Type',
       u'Description', u'Location_Description', u'Arrest', u'Domestic',
       u'Beat', u'District', u'Ward', u'Community_Area', u'FBI_Code',
       u'X_Coordinate', u'Y_Coordinate', u'Year', u'Updated_On', u'Latitude',
       u'Longitude', u'Location'],
      dtype='object')

In [101]:
crime_data['crime_2012']['Date'][:10]

0    05/01/2012 12:00:00 PM
1    06/01/2012 04:50:00 PM
2    11/01/2012 10:05:00 PM
3    01/01/2012 12:01:00 AM
4    11/01/2012 12:00:00 AM
5    06/16/2012 04:30:00 AM
6    11/21/2012 07:30:00 AM
7    05/18/2012 06:38:00 AM
8    03/26/2012 06:15:00 PM
9    08/07/2012 01:40:00 PM
Name: Date, dtype: object

### Add a column for the year for storage

In [37]:
for file_name in crime_data.keys():
    crime_data[file_name]['Year'] = int(file_name[-4:])

In [38]:
crime_data.keys()

['crime_2012', 'crime_2013', 'crime_2016', 'crime_2014', 'crime_2015']

In [39]:
crime_data['crime_2013'].shape

(306761, 22)

In [40]:
def create_db(db_path):
    con = sqlite3.connect(db_path)#Create database to store movie and rating info for easy storage and querying
    cur = con.cursor()
    try:    
        cur.execute("DROP TABLE IF EXISTS crime")#drop if already exists tables in DB
        cur.execute("""CREATE TABLE crime
       
                   ( ID                        integer,
                    Case_Number              text,
                    Date                     DATETIME,
                    Block                    text,
                    IUCR                     text,
                    Primary_Type             text,
                    Description              text,
                    Location_Description     text,
                    Arrest                     bool,
                    Domestic                   bool,
                    Beat                      integer,
                    District                numeric,
                    Ward                    numeric,
                    Community_Area          numeric,
                    FBI_Code                 object,
                    X_Coordinate            numeric,
                    Y_Coordinate            numeric,
                    Year                      integer,
                    Updated_On               object,
                    Latitude                numeric,
                    Longitude               numeric,
                    Location                 text

                   )
       
        """)
    except Exception as e:
        print(e)
    con.commit()
    con.close()

In [41]:
table_cols = [  'ID',
                'Case_Number'              ,
                'Date'                     ,
                'Block'                    ,
                'IUCR'                     ,
                'Primary_Type'             ,
                'Description'              ,
                'Location_Description'     ,
                'Arrest'                     ,
                'Domestic'                   ,
                'Beat'                      ,
                'District'                ,
                'Ward'                    ,
                'Community_Area'          ,
                'FBI_Code'                 ,
                'X_Coordinate'            ,
                'Y_Coordinate'            ,
                'Year'                      ,
                'Updated_On'               ,
                'Latitude'                ,
                'Longitude'               ,
                'Location' ]

#table_cols = ['dept_name' , 'emp_name', 'title' , 'jan' , 'feb' ,'mar' , 'apr' , 'may' , 'jun' ,'jul' , 'aug' , 'sep' , 'oct' , 'nov' , 'dec' , 'total' , 'year']

In [42]:
len(table_cols)

22

In [44]:
db_path = 'chicago_data.db'
create_db(db_path)

In [46]:
con = sqlite3.connect(db_path)#Create database to store movie and rating info for easy storage and querying
cur = con.cursor()

for file_name in crime_data.keys():
    try: 
        crime_data[file_name].columns = table_cols
    except:
        print(file_name)
    crime_data[file_name].to_sql('crime', con, if_exists='append', index=False)

Testing

In [64]:
con = sqlite3.connect(db_path)#Create database to store movie and rating info for easy storage and querying
cur = con.cursor()

In [69]:
cur.execute("""SELECT COUNT(*) FROM ot""")

In [70]:
cur.fetchall()

[(121097,)]

In [68]:
cur.execute("PRAGMA table_info(ot);")
print(cur.fetchall())


[(0, u'dept_name', u'text', 0, None, 0), (1, u'emp_name', u'text', 0, None, 0), (2, u'title', u'text', 0, None, 0), (3, u'jan', u'numeric', 0, None, 0), (4, u'feb', u'numeric', 0, None, 0), (5, u'mar', u'numeric', 0, None, 0), (6, u'apr', u'numeric', 0, None, 0), (7, u'may', u'numeric', 0, None, 0), (8, u'jun', u'numeric', 0, None, 0), (9, u'jul', u'numeric', 0, None, 0), (10, u'aug', u'numeric', 0, None, 0), (11, u'sep', u'numeric', 0, None, 0), (12, u'oct', u'numeric', 0, None, 0), (13, u'nov', u'numeric', 0, None, 0), (14, u'dec', u'numeric', 0, None, 0), (15, u'total', u'numeric', 0, None, 0), (16, u'year', u'integer', 0, None, 0)]


In [67]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[(u'ot',), (u'crime',)]


In [71]:
cur.close()
con.close()

- 'dept_name'
- 'emp_name'
- 'title'
- 'jan'
- 'feb'
- 'mar'
- 'apr'
- 'may'
- 'jun'
- 'jul'
- 'aug' 
- 'sep'
- 'oct'
- 'nov'
- 'dec'
- 'total' 
- 'year'

In [55]:
f = cur.execute("""SELECT total, year 
                FROM ot
                GROUP BY year""")

In [50]:
q = """SELECT total, year 
                FROM ot
                GROUP BY year"""

In [56]:
f.fetchall()

[(15554, 2012),
 (609.3, 2013),
 (50211.49, 2014),
 (22773.31, 2015),
 (25249.6, 2016)]

In [57]:
cur.close()

In [54]:
q

'SELECT total, year \n                FROM ot\n                GROUP BY year'

In [51]:
df = pd.read_sql(q,con)

In [52]:
df.head()

,total,year
0,15554.00,2012
1,609.30,2013
2,50211.49,2014
3,22773.31,2015
4,25249.60,2016


In [53]:
x= 0
for i in crime_data.keys():
    print(crime_data[i].shape[0])
    x += crime_data[i].shape[0]
x

335738
306761
267453
274666
263231


1447849

In [45]:
cur.close()